# Паттерны проектирования

### Классификация паттернов проектирования

Изначально существовало две основные классификации паттернов проектирования:

1. Какую проблему решает паттерн.
2. Как относится паттерн к классам или объектам.там.

Принимая во внимание первую классификацию, паттерны можно разделить на три группы:

* Порождающие – предоставляют возможность создания контролируемым образом, инициализации и конфигурации объектов, классов и типов данных на основе требуемых критериев.
* Структурные – помогают организовать структуры связанных объектов и классов, предоставляя новые функциональные возможности.
* Поведенческие – направлены на выявление общих моделей взаимодействия между объектами.

## Паттерн 1: Синглтон - Creational Patterns

In [4]:
# НАИВНЫЙ ПОДХОД С НАРУШЕНИЕМ ПРИНЦИПА ЕДИННОЙ ОТВЕТСТВЕННОСТИ
# class Logger:
#     @staticmethod
#     def get_instance():
#         if '_instance' not in Logger.__dict__:
#             Logger._instance = Logger()

#         return Logger._instance

#     def write_log(self, path):
#         pass

# s1 = Logger.get_instance()
# s2 = Logger.get_instance()

# assert s1 is s2

In [24]:
# ПРАВИЛЬНЫЙ ПОДХОД

# Наследование

class Singleton1:
    _instances = {}

    def __new__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__new__(cls)
            cls._instances[cls] = instance
        return cls._instances[cls]

class Logger1(Singleton1):
  def __init__(self, name):
      self.name = name
  def write_log(self, path):
      pass


s11 = Logger1(name='main')
s22 = Logger1(name='routers')
print(s11 is s22)
assert s11 is s22


# Метаклассы
class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class Logger(metaclass=Singleton):
  def __init__(self, name):
      self.name = name
  def write_log(self, path):
      pass

s1 = Logger(name='main')
s2 = Logger(name='routers')

print(s1 is s2)
assert s1 is s2

True
True


# Разновидность синглтона с разделяемым(shared) состоянием

In [54]:
class Borg:
    shared_state: dict[str, str] = {}

    def __init__(self):
        self.__dict__ = self.shared_state



class YourBorg(Borg):
    def __init__(self, state: str = None) -> None:
        super().__init__()
        if state:
            self.state = state
        else:
            # initiate the first instance with default state
            if not hasattr(self, "state"):
                self.state = "Init"

    def __str__(self) -> str:
        return self.state

        

In [55]:
def main():
    """
    >>> rm1 = YourBorg()
    >>> rm2 = YourBorg()

    >>> rm1.state = 'Idle'
    >>> rm2.state = 'Running'

    >>> print('rm1: {0}'.format(rm1))
    rm1: Running
    >>> print('rm2: {0}'.format(rm2))
    rm2: Running

    # When the `state` attribute is modified from instance `rm2`,
    # the value of `state` in instance `rm1` also changes
    >>> rm2.state = 'Zombie'

    >>> print('rm1: {0}'.format(rm1))
    rm1: Zombie
    >>> print('rm2: {0}'.format(rm2))
    rm2: Zombie

    # Even though `rm1` and `rm2` share attributes, the instances are not the same
    >>> rm1 is rm2
    False

    # New instances also get the same shared state
    >>> rm3 = YourBorg()

    >>> print('rm1: {0}'.format(rm1))
    rm1: Zombie
    >>> print('rm2: {0}'.format(rm2))
    rm2: Zombie
    >>> print('rm3: {0}'.format(rm3))
    rm3: Zombie

    # A new instance can explicitly change the state during creation
    >>> rm4 = YourBorg('Running')

    >>> print('rm4: {0}'.format(rm4))
    rm4: Running

    # Existing instances reflect that change as well
    >>> print('rm3: {0}'.format(rm3))
    rm3: Running
    """

import doctest
doctest.testmod()

TestResults(failed=0, attempted=17)

# Паттерн 2 - Абстрактная фабрика  - Creational patterns

### TL;DR
Provides a way to encapsulate a group of individual factories.

In [62]:
class Pet:
    def __init__(self, name: str) -> None:
        self.name = name

    def speak(self) -> None:
        raise NotImplementedError

    def __str__(self) -> str:
        raise NotImplementedError


class Dog(Pet):
    def speak(self) -> None:
        print("woof")

    def __str__(self) -> str:
        return f"Dog <{self.name}>"


class Cat(Pet):
    def speak(self) -> None:
        print("meow")

    def __str__(self) -> str:
        return f"Cat <{self.name}>"
        

In [63]:
from typing import Type

class PetShop:
    """A pet shop"""

    def __init__(self, animal_factory: Type[Pet]) -> None:
        """pet_factory is our abstract factory.  We can set it at will."""
        self.pet_factory = animal_factory

    def buy_pet(self, name: str) -> Pet:
        """Creates and shows a pet using the abstract factory"""
        pet = self.pet_factory(name)
        print(f"Here is your lovely {pet}")
        return pet


In [65]:
pet_factory = PetShop(Dog)
awesome_boy = pet_factory.buy_pet('woofie^_^')
awesome_boy.speak()
print(awesome_boy)

Here is your lovely Dog <woofie^_^>
woof
Dog <woofie^_^>


# Паттерн 3 - ФАБРИКА/Фабричный метод - Creational patterns

#### *TL;DR
Creates objects without having to specify the exact class.

from typing import Protocol


class Localizer(Protocol):
    def localize(self, msg: str) -> str:
        pass


class GreekLocalizer:
    """ A simple localizer a la gettext """
    def __init__(self) -> None:
        self.translations = {"dog": "σκύλος", "cat": "γάτα"}

    def localize(self, msg: str) -> str:
        """We'll punt if we don't have a translation"""
        return self.translations.get(msg, msg)


class EnglishLocalizer:
    """Simply echoes the message"""

    def localize(self, msg: str) -> str:
        return msg


def get_localizer(language: str = "English") -> Localizer:

    """Factory"""
    localizers: dict[str, Type[Localizer]] = {
        "English": EnglishLocalizer,
        "Greek": GreekLocalizer,
    }

    return localizers[language]()


In [71]:
gr_l = get_localizer('Greek')
print(gr_l.localize('dog'))
print(gr_l.localize('cat'))


en_l = get_localizer('English')
print(en_l.localize('dog'))
print(en_l.localize('HULIO IGLESIAS'))

σκύλος
γάτα
dog
HULIO IGLESIAS


# Паттерн 4 Стратегия - Behavioral Patterns

#### *TL;DR
Enables selecting an algorithm at runtime.

In [91]:
from __future__ import annotations

from typing import Callable


class DiscountStrategyValidator:
    @staticmethod
    def validate(obj: Order, value: Callable) -> bool:
        try:
            if obj.price - value(obj) < 0:
                raise ValueError(
                    f"Discount cannot be applied due to negative price resulting. {value.__name__}"
                )
        except ValueError as ex:
            print(str(ex))
            return False
        else:
            return True

    def __set_name__(self, owner, name: str) -> None:
        self.private_name = f"_{name}"

    def __set__(self, obj: Order, value: Callable = None) -> None:
        if value and self.validate(obj, value):
            setattr(obj, self.private_name, value)
        else:
            setattr(obj, self.private_name, None)

    def __get__(self, obj: object, objtype: type = None):
        return getattr(obj, self.private_name)


class Order:
    discount_strategy = DiscountStrategyValidator()

    def __init__(self, price: float, discount_strategy: Callable = None) -> None:
        self.price: float = price
        self.discount_strategy = discount_strategy

    def apply_discount(self) -> float:
        if self.discount_strategy:
            discount = self.discount_strategy(self)
        else:
            discount = 0

        return self.price - discount

    def __repr__(self) -> str:
        return f"<Order price: {self.price} with discount strategy: {getattr(self.discount_strategy,'__name__',None)}>"


def ten_percent_discount(order: Order) -> float:
    return order.price * 0.10


def on_sale_discount(order: Order) -> float:
    return order.price * 0.25 + 20


order = Order(100, discount_strategy=ten_percent_discount)
print(order)
print(order.apply_discount())


order = Order(100, discount_strategy=on_sale_discount)
print(order)
print(order.apply_discount())


order = Order(10, discount_strategy=on_sale_discount)
print(order)

<Order price: 100 with discount strategy: ten_percent_discount>
90.0
<Order price: 100 with discount strategy: on_sale_discount>
55.0
Discount cannot be applied due to negative price resulting. on_sale_discount
<Order price: 10 with discount strategy: None>


In [93]:
# LIGHTWEIGHT SOLUTION 

class Item:
 
    """Constructor function with price and discount"""
 
    def __init__(self, price, discount_strategy = None):
         
        """take price and discount strategy"""
         
        self.price = price
        self.discount_strategy = discount_strategy
         
    """A separate function for price after discount"""
 
    def price_after_discount(self):
         
        if self.discount_strategy:
            discount = self.discount_strategy(self)
        else:
            discount = 0
             
        return self.price - discount
 
    def __repr__(self):
         
        statement = "Price: {}, price after discount: {}"
        return statement.format(self.price, self.price_after_discount())
 
"""function dedicated to On Sale Discount"""
def on_sale_discount(order):
     
    return order.price * 0.25 + 20
 
"""function dedicated to 20 % discount"""
def twenty_percent_discount(order):
     
    return order.price * 0.20
 
 
print(Item(20000))
 
"""with discount strategy as 20 % discount"""
print(Item(20000, discount_strategy = twenty_percent_discount))

"""with discount strategy as On Sale Discount"""
print(Item(20000, discount_strategy = on_sale_discount))

Price: 20000, price after discount: 20000
Price: 20000, price after discount: 16000.0
Price: 20000, price after discount: 14980.0


In [109]:
from typing import Callable

class Order:
    def __init__(self, price, discount_strategy: Callable = None):
        self.price = price
        self.discount_strategy = discount_strategy

    def calculate_discount(self):
        if self.discount_strategy:
            return self.discount_strategy(self)
        else:
            return self.price

    def __str__(self):
        return f'Final price with discount is {self.calculate_discount()}'

def season_regular_sale(order):
    # 25% off
    return order.price * 0.75


def winter_crazy_sale(order):
    # 60% off
    return order.price * 0.40

season_sale_order = Order(100, season_regular_sale)
print(season_sale_order)

winter_crazy_sale_order = Order(100, winter_crazy_sale)
print(winter_crazy_sale_order)

# Паттерн 5 Цепочка обязанностей - Behavioral Patterns

#### *TL;DR
The Chain of responsibility is an object oriented version of the
`if ... elif ... elif ... else ...` idiom, with the
benefit that the condition–action blocks can be dynamically rearranged
and reconfigured at runtime.

Allow a request to pass down a chain of receivers until it is handled.

In [115]:
from abc import ABC, abstractmethod
from typing import Optional, Tuple


class Handler(ABC):
    def __init__(self, successor: Optional["Handler"] = None):
        self.successor = successor

    def handle(self, request: int) -> None:
        """
        Handle request and stop.
        If can't - call next handler in chain.

        As an alternative you might even in case of success
        call the next handler.
        """
        res = self.check_range(request)
        if not res and self.successor:
            self.successor.handle(request)

    @abstractmethod
    def check_range(self, request: int) -> Optional[bool]:
        """Compare passed value to predefined interval"""


class ConcreteHandler0(Handler):
    """Each handler can be different.
    Be simple and static...
    """

    @staticmethod
    def check_range(request: int) -> Optional[bool]:
        if 0 <= request < 10:
            print(f"request {request} handled in handler 0")
            return True
        return None


class ConcreteHandler1(Handler):
    """... With it's own internal state"""

    start, end = 10, 20

    def check_range(self, request: int) -> Optional[bool]:
        if self.start <= request < self.end:
            print(f"request {request} handled in handler 1")
            return True
        return None


class ConcreteHandler2(Handler):
    """... With helper methods."""

    def check_range(self, request: int) -> Optional[bool]:
        start, end = self.get_interval_from_db()
        if start <= request < end:
            print(f"request {request} handled in handler 2")
            return True
        return None

    @staticmethod
    def get_interval_from_db() -> Tuple[int, int]:
        return (20, 30)

class FallbackHandler(Handler):
    @staticmethod
    def check_range(request: int) -> Optional[bool]:
        print(f"end of chain, no handler for {request}")
        return False

In [116]:
h0 = ConcreteHandler0()
h1 = ConcreteHandler1()
h2 = ConcreteHandler2(FallbackHandler())
h0.successor = h1
h1.successor = h2

requests = [2, 5, 14, 22, 18, 3, 35, 27, 20]
for request in requests:
    h0.handle(request)

request 2 handled in handler 0
request 5 handled in handler 0
request 14 handled in handler 1
request 22 handled in handler 2
request 18 handled in handler 1
request 3 handled in handler 0
end of chain, no handler for 35
request 27 handled in handler 2
request 20 handled in handler 2


# Паттерн 6 Адаптер - Structural Patterns

#### *TL;DR
Allows the interface of an existing class to be used as another interface.

In [1]:
class Dog:
    def __init__(self) -> None:
        self.name = "Dog"

    def bark(self) -> str:
        return "woof!"


class Cat:
    def __init__(self) -> None:
        self.name = "Cat"

    def meow(self) -> str:
        return "meow!"


class Human:
    def __init__(self) -> None:
        self.name = "Human"

    def speak(self) -> str:
        return "'hello'"


class Car:
    def __init__(self) -> None:
        self.name = "Car"

    def make_noise(self, octane_level: int) -> str:
        return f"vroom{'!' * octane_level}"

In [10]:
from typing import Callable, TypeVar


T = TypeVar("T")


class Adapter:
    """Adapts an object by replacing methods.

    Usage
    ------
    dog = Dog()
    dog = Adapter(dog, make_noise=dog.bark)
    """

    def __init__(self, obj: T, **adapted_methods: Callable):
        """We set the adapted methods in the object's dict."""
        self.obj = obj
        self.__dict__.update(adapted_methods)

    def __getattr__(self, attr):
        """All non-adapted calls are passed to the object."""
        return getattr(self.obj, attr)

    def original_dict(self):
        """Print original object dict."""
        return self.obj.__dict__

In [14]:
objects = []
dog = Dog()
print(dog.__dict__)

{'name': 'Dog'}


In [15]:
objects.append(Adapter(dog, make_noise=dog.bark))

In [16]:
cat = Cat()
objects.append(Adapter(cat, make_noise=cat.meow))

In [17]:
human = Human()
objects.append(Adapter(human, make_noise=human.speak))

In [18]:
car = Car()
objects.append(Adapter(car, make_noise=lambda: car.make_noise(3)))

In [19]:
for obj in objects:
    print("A {0} goes {1}".format(obj.name, obj.make_noise()))

A Dog goes woof!
A Cat goes meow!
A Human goes 'hello'
A Car goes vroom!!!


# Паттерн 7 Фасад - Structural Patterns

#### *TL;DR
Provides a simpler unified interface to a complex system.

In [21]:
# Complex computer parts
class CPU:
    """
    Simple CPU representation.
    """

    def freeze(self) -> None:
        print("Freezing processor.")

    def jump(self, position: str) -> None:
        print("Jumping to:", position)

    def execute(self) -> None:
        print("Executing.")


In [22]:
class Memory:
    """
    Simple memory representation.
    """

    def load(self, position: str, data: str) -> None:
        print(f"Loading from {position} data: '{data}'.")

In [23]:
class SolidStateDrive:
    """
    Simple solid state drive representation.
    """

    def read(self, lba: str, size: str) -> str:
        return f"Some data from sector {lba} with size {size}"

In [24]:
class ComputerFacade:
    """
    Represents a facade for various computer parts.
    """

    def __init__(self):
        self.cpu = CPU()
        self.memory = Memory()
        self.ssd = SolidStateDrive()

    def start(self):
        self.cpu.freeze()
        self.memory.load("0x00", self.ssd.read("100", "1024"))
        self.cpu.jump("0x00")
        self.cpu.execute()

In [25]:
computer_facade = ComputerFacade()
computer_facade.start()

Freezing processor.
Loading from 0x00 data: 'Some data from sector 100 with size 1024'.
Jumping to: 0x00
Executing.
